In [24]:
import numpy as np
import pandas as pd

In [25]:
classesDict = {
    "T1":0,
    "T1_c":1,
    "T2":2,
    "T2-FLAIR":3,
    "FS":4,
    "PD":5,
    "SWI":6,
    "GRE":7,
    "T2*":8,
    "DWI":9,
    "ADC":10,
    "BOLD":11,
    "FlowSensitve":12,
    "PWI":13,
    "ASL":14,
    "Hippo":15,
    "DTI":16,
    "Other":17
}

In [26]:
labels_path = '/home/soenke/Internship/DeepDicomSortFork/prototyping/CombinedTrainingFrame_A2_A3_O3_egd.txt'

In [27]:
Labels = pd.read_csv(labels_path,names=['ID','label','extra'],sep='\t', dtype={'ID':str,'label':int,'extra':int})

In [28]:
Label_name_list = list(classesDict.keys())

In [29]:
Labels

,ID,label,extra
0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0
1,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0
2,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,8,0
3,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,15,0
4,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,0,0
...,...,...,...
908020,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0
908021,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0
908022,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0
908023,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0


In [30]:
dataset = []
Label_txt = []
for idx in Labels.index:
    ID = Labels['ID'][idx]
    label = Labels['label'][idx]
    Label_txt.append(Label_name_list[label])
    if 'ADNI2' in ID:
        dataset.append('ADNI2')
        continue
    elif 'ADNI3' in ID:
        dataset.append('ADNI3')
        continue
    elif 'OASIS3' in ID:
        dataset.append('OASIS3')
        continue
    elif 'egd' in ID:
        dataset.append('egd')
    

In [31]:
temp = [(d, l) for d,l in zip(dataset, Label_txt)]

In [32]:
extraInfoFrame = pd.DataFrame(temp, columns=['dataset', 'label_txt'])

In [33]:
FullLabelFrame = pd.concat([Labels, extraInfoFrame], axis=1)

In [34]:
FullLabelFrame

,ID,label,extra,dataset,label_txt
0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0,ADNI3,T2-FLAIR
1,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,3,0,ADNI3,T2-FLAIR
2,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,8,0,ADNI3,T2*
3,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,15,0,ADNI3,Hippo
4,/trinity/home/r098375/DDS/data/ADNI/ADNI3/trai...,0,0,ADNI3,T1
...,...,...,...,...,...
908020,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,egd,DWI
908021,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,egd,DWI
908022,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,egd,DWI
908023,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,9,0,egd,DWI


In [36]:
FullLabelFrame.groupby('dataset')['label_txt'].value_counts(normalize=True)

dataset  label_txt   
ADNI2    PD              0.500000
         T2              0.500000
ADNI3    Other           0.330598
         T2*             0.227574
         T1              0.131810
         Hippo           0.115018
         T2-FLAIR        0.113440
         PWI             0.055173
         ASL             0.020706
         DTI             0.004671
         SWI             0.000884
         PD              0.000126
OASIS3   Other           0.258561
         BOLD            0.145994
         T2              0.121154
         T1              0.117404
         DWI             0.097456
         T2*             0.068975
         ASL             0.045222
         GRE             0.042287
         SWI             0.039733
         T2-FLAIR        0.036743
         FlowSensitve    0.026470
egd      T1_c            0.201422
         DWI             0.198460
         T1              0.151659
         T2              0.136256
         PWI             0.124408
         T2-FLAIR        0